# Image processing: Contact angle measurement and line tracking for Novec
This notebook contains the image processing steps for Novec (Section 2.1.2), including
- kMeans clustering for interface detection
- RANSAC fit of interface and contact angle measurement (step iii)
- interface front and back tracking (step iv).

Although the general functionalities for contact angle measurement and line tracking are the same as for water and Tween,\
Novec requires extended postprocessing, such as the kMeans clustering and separate tracking of the interface front and back.\
Therefore, it is processed in a separate file for clarity.

In [ ]:
import cv2
import numpy as np
import math
import matplotlib.pyplot as plt
from matplotlib import colormaps
import os
from os.path import join
import pickle
import pandas as pd
from sklearn import cluster
from sklearn.linear_model import RANSACRegressor
from sklearn.metrics import mean_squared_error

from shared_functions import *

## Functions

In [ ]:
def run_kmeans_grayscale(img_gray):
    """
    Performs KMeans clustering with 2 clusters on a grayscale image to identify the interface.
    The cluster containing less pixels is chosen as the interface.
    
    Args:
    img_gray (numpy.ndarray): input grayscale image on which KMeans clustering will be performed.

    Returns:
    numpy.ndarray: binary mask representing the identified interface.
    """
    
    # reshape image for clustering
    x, y = img_gray.shape
    X = img_gray.reshape(x*y).reshape(-1, 1)

    # clustering
    km = cluster.KMeans(n_clusters=2, init='k-means++', n_init=10, random_state = 42).fit(X)

    # get labels
    labels = km.labels_.reshape(x,y)
    
    # create mask for interface
    mask = labels > 0.1

    # revert mask if reversed
    if len(np.argwhere(mask)) > 0.5 * len(mask) *  len(mask[0]):
        mask = labels < 0.1
    
    return mask

In [ ]:
def find_front_and_back_of_row_in_mask(row_):
    """
    Finds the front and back indices of a non-empty row in a mask.
    
    Args:
    row_ (list): input row represented as a list, containing 1 (interface pixels) and 0 (packground pixels).

    Returns:
    tuple containing the indices of the front and back of the interface in the row.
    """

    # if row is not empty (contains an interface)
    if any(row_):
        
        # find back pixel (first occurrance of 1)
        idx_back = [idx for idx, x in enumerate(row_) if x][0]

        # find front pixel (last occurance of 1 before next 0 in row,
        # if no front is found, take last index of row)
        if len([idx for idx, x in enumerate(row_[idx_back:]) if ~x])>0:
            idx_front = idx_back + [idx for idx, x in enumerate(row_[idx_back:]) if ~x][0]
        else:
            idx_front = len(row_)
            
    # if row is emtpy: set dummy values
    else:
        idx_front = 0
        idx_back = 0
        
    return idx_front, idx_back

In [ ]:
def create_estimator_object(fit_degree):
    """
    Creates an instance of the PolynomialRegression class with the specified degree for polynomial fitting.
    This wrapper function is necessary to ensure the right fitting degree.
    In sklearn 1.2.2 it is not possible to specify the correct fit degree in ransac.fit, 
    even though the instance was created with the correct degree.
    Therefore this class works as a workaround to force the degree.

    Args:
    fit_degree (int): degree for polynomial fitting.

    Returns:
    An instance of the PolynomialRegression class with the specified degree.
    """
    class PolynomialRegression(object):
        def __init__(self, degree=fit_degree, coeffs=None):
            self.degree = degree
            self.coeffs = coeffs

        def fit(self, X, y):
            self.coeffs = np.polyfit(X.ravel(), y, self.degree)

        def get_params(self, deep=True):
            return {'coeffs': self.coeffs}

        def set_params(self, coeffs=None, random_state=None):
            self.coeffs = coeffs

        def predict(self, X):
            poly_eqn = np.poly1d(self.coeffs)
            y_result = poly_eqn(X.ravel())
            return y_result

        def score(self, X, y):
            return mean_squared_error(y, self.predict(X))
        
    return PolynomialRegression()

In [ ]:
def fit_interface_RANSAC(x_vals, y_vals, polyfit_degree, residual_threshold, min_samples):
    """
    Fits the interface pixels using the RANSAC algorithm with polynomial regression.

    Args:
    x_vals (numpy.ndarray): x-values of the data points.
    y_vals (numpy.ndarray): y-values of the data points.
    polyfit_degree (int): degree for polynomial fitting.
    residual_threshold (float): maximum residual for a data point to be classified as an inlier.
    min_samples (int): The minimum number of samples required to fit the model.

    Returns:
    tuple: A tuple containing the following elements:
     - y_result (numpy.ndarray): predicted y-values of the contact line.
     - inlier_mask (numpy.ndarray): boolean mask indicating the inliers.
     - theta_top_deg (float): resulting contact angle at the interface top.
     - theta_bot_deg (float): resulting contact angle at the interface bottom.
    """

    # create regression object 
    estimator = create_estimator_object(polyfit_degree)
    ransac = RANSACRegressor(estimator,
                             residual_threshold= residual_threshold, 
                             random_state=0,
                             min_samples=min_samples)

    # fit data
    ransac.fit(np.expand_dims(x_vals, axis=1), y_vals)
    inlier_mask = ransac.inlier_mask_
    y_result = ransac.predict(np.expand_dims(x_vals, axis=1))
    params = ransac.estimator_.coeffs
    
    # get the slope of the fitted polynomial at the wall
    fitted_polynomial = np.poly1d(params)
    derivative_of_fittted_polynomial = fitted_polynomial.deriv() 
    points_of_polynomial_derivative = np.polyval(derivative_of_fittted_polynomial.c, np.sort(x_vals[inlier_mask]))
    
    # get the slope at the walls
    slope_top = points_of_polynomial_derivative[0]
    slope_bot = points_of_polynomial_derivative[-1]
    
    # measure contact angle      
    theta_top_deg = math.degrees(math.atan(slope_top))
    theta_bot_deg = math.degrees(math.atan(slope_bot))

    return y_result, inlier_mask, theta_top_deg, theta_bot_deg

## KMeans Clustering to extract interface

**User input: selection of cases and general parameters**

In [ ]:
# choose fluid
fluid =  "03_novec"

# choose cases that should be considered
# if empty --> all cases are used. insert 1,2,3 to choose case 1,2,3
case_selection = [] 

# choose the step of processed images (1 --> every image is used)
step_images = 1

# choose which result to plot ("all", "last", "none") --> "all" leads to longer runime.
plot_pictures = "last"

# choose at which point to end the analysis.
# use "when_cavities_are_reached" for contact angle measurements, "at_final_image" for line tracking.
end_analysis = "when_cavities_are_reached"

# choose paths
path_data_experiments = "../data_experiments"
path_df_postproc = join(path_data_experiments, "case_parameters.xlsx")
path_df_channel_edges = join(path_data_experiments, fluid, "02_channel_edges", "df_channel_edges.csv")
path_images = join(path_data_experiments, fluid, "01_images_preprocessed")
path_save = join(path_data_experiments, fluid, "03_contact_angle_measurement")

# create directory for saving results and plots
os.makedirs(path_save, exist_ok=True)

**Run the analysis**

In [ ]:
# set case selection based on user imput
if case_selection == []:
    cases = [int(file) for file in os.listdir(path_images) if len(file)<3]
else: 
    cases = case_selection
print(f"selected cases: {cases}")

# initialize lists for results
mask_interface_all = []

# loop over cases
for case in cases:
    print(f'Case {case}')
 
    # get case data
    x_start, x_cavities, y_channelEdge_bottom, y_channelEdge_top, framerate, selected_images = get_case_parameters(fluid, case, path_df_channel_edges, path_df_postproc, path_images, step_images, end_analysis)    
    print(f'\t number of selected images: {len(selected_images)}')

    # initialize list for storing interface masks
    mask_interface_case = [ [] for i in range(len(selected_images)) ]

    # loop over images
    for i in range(len(selected_images)):
        
        # ----- Get Image and ROI

        # Load the image and convert it to grayscale
        img_name = selected_images[i]
        image = cv2.imread(join(path_images, str(case), img_name))
        image_gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

        # Region of interest of image
        x_roi = [x_start, len(image_gray[0])-x_start]
        y_roi = [y_channelEdge_top+1,y_channelEdge_bottom-1]
        image_roi = image_gray[y_roi[0]:y_roi[1] , x_roi[0]:x_roi[1]]

        # ----- Run KMeans in ROI

        mask_interface_case[i] = run_kmeans_grayscale(image_roi)

        # ----- Plot
        # check whether to plot now
        plot_now = check_if_plot_now(plot_pictures, selected_images, i)

        if plot_now == True:
            
            fig, (ax1, ax2) = plt.subplots(ncols=2, figsize=(10,4))

            # plot image
            plt_img = ax1.imshow(image_roi, cmap='gray', vmin=0, vmax=255)
            fig.colorbar(plt_img, ax=ax1, location='top', anchor=(0.5, 0), shrink=0.6)

            # plot kMeans result
            plt_km = ax2.imshow(mask_interface_case[i], cmap=colormaps.get_cmap('Pastel1_r').resampled(2))
            fig.colorbar(plt_km, ax=ax2, location='top', anchor=(0.5,0), shrink=0.6, ticks=[0,1])

            # cosmetics
            ax1.set_title(f'case {case}, image {img_name[-8:-4]}', y=1.21)
            ax2.set_title('KMeans', y=1.21)

            # save figure
            #fig.savefig(os.path.join(path_save, f"kMeans_{fluid}_{case}_{img_name[-8:-4]}.png"), bbox_inches='tight', dpi=300)
        
    # collect data from cases
    mask_interface_all.append(mask_interface_case)

**save kmeans data**

In [ ]:
# create dataframe
df_kmeans = pd.DataFrame(zip(mask_interface_all), index=cases, columns =['mask_interface'])

#display datarame
display(df_kmeans)

# save dataframe 
df_kmeans.to_pickle(os.path.join(path_data_experiments, fluid, f"df_kmeans_{fluid}.pickle"))

## (iii) RANSAC fit and contact angle measurement

**User input: selection of cases and general parameters**

In [ ]:
# choose fluid
fluid =  "03_novec"

# choose cases that should be considered
# if empty --> all cases are used. insert 1,2,3 to choose case 1,2,3
case_selection = []  

# choose the step of processed images (1 --> every image is used)
step_images = 1

# choose which result to plot ("all", "last", "none") --> "all" leads to longer runime.
plot_pictures = "last"

# choose at which point to end the analysis.
# use "when_cavities_are_reached" for contact angle measurements, "at_final_image" for line tracking.
end_analysis = "when_cavities_are_reached"

# choose paths
path_data_experiments = "../data_experiments"
path_df_postproc = join(path_data_experiments, "case_parameters.xlsx")
path_df_channel_edges = join(path_data_experiments, fluid, "02_channel_edges", "df_channel_edges.csv")
path_df_kmeans = join(path_data_experiments, fluid,f"df_kmeans_{fluid}.pickle")
path_images = join(path_data_experiments, fluid, "01_images_preprocessed")
path_save = join(path_data_experiments, fluid, "03_contact_angle_measurement")

# load kmeans dataframe (load it only once since it can take some time)
df_kmeans = pd.read_pickle(path_df_kmeans)

# create directory for saving results and plots
os.makedirs(path_save, exist_ok=True)

**User input: RANSAC fitting parameters** \
Two different configurations are used, one for low contact angles (u<1e-2 m/s), one for high contact angles (u=1e-2m/s). \
In constrast to water and Tween, the processing for Novec requires no pre-filtering of outliers because the interface is well distinguishable in all images after clustering.
- fit_degree (int): degree for polynomial fitting.
- fit_method (str): method for curve fitting, either 'fit_y(x)' or 'fit_x(y)'. \
    For low contact angles, a fit y(x) is used, for larger contact angles, x(y) gives better results. \
    The respective fitting methods also use different fit parameters.
- cutoff (int): cutoff value specifiying how many pixels are considered for the curve fit.
- fit_degree (int): degree for polynomial fitting.
- min_samples (int): minimum amount of considered inliers.
- residual_threshold_front (int): maximum residual for a data point to be classified as an inlier during RANSAC curve fitting of the interface front.
- residual_threshold_back (int): maximum residual for a data point to be classified as an inlier during RANSAC curve fitting of the interface back.


In [ ]:
fit_degree = 3

ransac_parameters = {
 
    "low_contact_angles": {

        "fit_method" : "fit_y(x)",
        "cutoff" : 50,
        "min_samples" : 20,
        "residual_threshold_front" : 10 ,
        "residual_threshold_back" : 5,      
        
    },
    
    "high_contact_angles": {
    
        "fit_method" : "fit_x(y)",
        "cutoff" : 100,
        "min_samples" : 20,
        "residual_threshold_front" : 10,
        "residual_threshold_back" : 10, 
    }
}

**Run the analysis**

In [ ]:
# set case selection based on user imput
if case_selection == []:
    cases = [int(file) for file in os.listdir(path_images) if len(file)<3]
else: 
    cases = case_selection
print(f"selected cases: {cases}")

#initialize lists for results
list_ca_ransac_front_top, list_ca_ransac_front_bot, list_rms_dist_to_fit_front_top_ransac, list_rms_dist_to_fit_front_bot_ransac = ([] for i in range(4))
list_ca_ransac_back_top, list_ca_ransac_back_bot, list_rms_dist_to_fit_back_top_ransac,list_rms_dist_to_fit_back_bot_ransac = ([] for i in range(4))

# loop over cases
for case in cases:
    print(f"Case {case}")
 
    # get case data
    x_start, x_cavities, y_channelEdge_bottom, y_channelEdge_top, framerate, selected_images = get_case_parameters(fluid, case, path_df_channel_edges, path_df_postproc, path_images, step_images, end_analysis)    
    print(f'\t number of selected images: {len(selected_images)}')
        
     # get interface cluster
    mask_interface_case = df_kmeans['mask_interface'][case][::step_images]

        
    # get fit parameters
    if (fluid == "03_novec" and case in [1,2,3,4,5])  \
        or ("u1e-2" in fluid): 
            fit_method, cutoff, min_samples, residual_threshold_front, residual_threshold_back = ransac_parameters["low_contact_angles"].values()
            
    elif (fluid == "03_novec" and case in [6,7,8,9,10,11,12,13]) \
        or ("u1e-1" in fluid): 
            fit_method, cutoff, min_samples, residual_threshold_front, residual_threshold_back = ransac_parameters["high_contact_angles"].values()
    
    else:
        raise Exception ("for this fluid, no ransac fitting parameters are defined")

    # initialize
    theta_ransac_front_top_case, theta_ransac_front_bot_case, rms_dist_to_fit_ransac_front_top_case, rms_dist_to_fit_ransac_front_bot_case = ([] for i in range(4)) 
    theta_ransac_back_top_case, theta_ransac_back_bot_case, rms_dist_to_fit_ransac_back_top_case, rms_dist_to_fit_ransac_back_bot_case = ([] for i in range(4))    
    
    # loop over images
    for i in range(len(selected_images)):
        #print(f'\t i = {i}')

        # ----- Get Image and ROI

        # Load the image and convert it to grayscale
        img_name = selected_images[i]
        image = cv2.imread(os.path.join(path_images, str(case), img_name))
        image_gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

        # Region of interest of image
        x_roi = [x_start, len(image_gray[0])-x_start]
        y_roi = [y_channelEdge_top+1,y_channelEdge_bottom-1]
        image_roi = image_gray[y_roi[0]:y_roi[1] , x_roi[0]:x_roi[1]]

    # ---------------- sample front and back
        
        # get interface mask for the image
        mask_interface = mask_interface_case[i][1:-2,:]
        
        # initialize
        x_front, x_back =  (np.empty(0) for _ in range(2))
        y_front, y_back = (np.array(range(len(mask_interface[:,0]))) for _ in range(2))
       
        # sample over rows
        for k in range(len(mask_interface)):
            row = mask_interface[k,:]

            # get front and back for row
            (idx_front, idx_back) = find_front_and_back_of_row_in_mask(row)

            x_front = np.append(x_front, idx_front - 1)
            x_back = np.append(x_back, idx_back)

        # cut off
        y_front_top = y_front[:cutoff]  
        x_front_top = x_front[:cutoff]  
        y_front_bot = y_front[-cutoff:]
        x_front_bot = x_front[-cutoff:]  
    
        y_back_top = y_back[:cutoff]  
        x_back_top = x_back[:cutoff]  
        y_back_bot = y_back[-cutoff:]  
        x_back_bot = x_back[-cutoff:]  
        
        # -------------- contact angle detection using RANSAC
                    
        # for large contact angles - fit x(y)
        if fit_method == 'fit_x(y)':
            
            # interface top (only for experimental images, becasue simulations are done only on bottom half of geometry)
            if not "sim" in fluid:
                
                # ransac fit
                (x_fit_front_top, inlier_mask_front_top, theta_front_top, trash) = fit_interface_RANSAC(y_front_top, x_front_top, fit_degree, residual_threshold_front, min_samples)
                (x_fit_back_top, inlier_mask_back_top, theta_back_top, trash) = fit_interface_RANSAC(y_back_top, x_back_top, fit_degree, residual_threshold_back, min_samples)

                # transform angles 
                theta_ransac_front_top_case.append(theta_front_top+90)
                theta_ransac_back_top_case.append(theta_back_top+90)

                # calculate rms distance between inliers and fit
                rms_dist_to_fit_ransac_front_top_case.append(np.sqrt(np.mean((x_fit_front_top[inlier_mask_front_top]-x_front_top[inlier_mask_front_top])**2)))
                rms_dist_to_fit_ransac_back_top_case.append(np.sqrt(np.mean((x_fit_back_top[inlier_mask_back_top]-x_back_top[inlier_mask_back_top])**2)))

            # repeat procedure for interface bottom
            (x_fit_front_bot, inlier_mask_front_bot, trash, theta_front_bot) = fit_interface_RANSAC(y_front_bot, x_front_bot, fit_degree, residual_threshold_front, min_samples)
            (x_fit_back_bot, inlier_mask_back_bot, trash, theta_back_bot) = fit_interface_RANSAC(y_back_bot, x_back_bot, fit_degree, residual_threshold_back, min_samples)

            theta_ransac_front_bot_case.append(90-theta_front_bot)  
            theta_ransac_back_bot_case.append(90-theta_back_bot)   
            
            rms_dist_to_fit_ransac_front_bot_case.append(np.sqrt(np.mean((x_fit_front_bot[inlier_mask_front_bot]-x_front_bot[inlier_mask_front_bot])**2)))
            rms_dist_to_fit_ransac_back_bot_case.append(np.sqrt(np.mean((x_fit_back_bot[inlier_mask_back_bot]-x_back_bot[inlier_mask_back_bot])**2)))

        # for small contact angles - fit y(x)
        elif  fit_method == 'fit_y(x)':
            
            # interface top (only for experimental images, becasue simulations are done only on bottom half of geometry)
            if not "sim" in fluid:
                
                (y_fit_front_top, inlier_mask_front_top, trash, theta_front_top) = fit_interface_RANSAC(x_front_top, y_front_top, fit_degree, residual_threshold_front, min_samples)
                (y_fit_back_top, inlier_mask_back_top, trash, theta_back_top) = fit_interface_RANSAC(x_back_top, y_back_top, fit_degree, residual_threshold_back, min_samples)

                theta_ransac_front_top_case.append(-theta_front_top)
                theta_ransac_back_top_case.append(-theta_back_top)

                rms_dist_to_fit_ransac_front_top_case.append(np.sqrt(np.mean((y_fit_front_top[inlier_mask_front_top]-y_front_top[inlier_mask_front_top])**2)))
                rms_dist_to_fit_ransac_back_top_case.append(np.sqrt(np.mean((y_fit_back_top[inlier_mask_back_top]-y_back_top[inlier_mask_back_top])**2)))
                
            # interface bottom
            (y_fit_front_bot, inlier_mask_front_bot, trash, theta_front_bot) = fit_interface_RANSAC(x_front_bot, y_front_bot, fit_degree, residual_threshold_front, min_samples)
            (y_fit_back_bot, inlier_mask_back_bot, trash, theta_back_bot) = fit_interface_RANSAC(x_back_bot, y_back_bot, fit_degree, residual_threshold_back, min_samples)

            theta_ransac_back_bot_case.append(theta_back_bot)    
            theta_ransac_front_bot_case.append(theta_front_bot)    
            
            rms_dist_to_fit_ransac_front_bot_case.append(np.sqrt(np.mean((y_fit_front_bot[inlier_mask_front_bot]-y_front_bot[inlier_mask_front_bot])**2)))
            rms_dist_to_fit_ransac_back_bot_case.append(np.sqrt(np.mean((y_fit_back_bot[inlier_mask_back_bot]-y_back_bot[inlier_mask_back_bot])**2)))
        
        # ----------------plot
        
        plot_now = check_if_plot_now(plot_pictures, selected_images, i)
        if plot_now == True:

            fig, ax = plt.subplots(figsize=(10,10))

            # plot image
            ax.imshow(image_roi, cmap='gray', vmin=0, vmax=255)

            # plot the front
            #ax.plot(x_front, y_front, '--b', linewidth=3, label = 'interface front and back')
            #ax.plot(x_back, y_back, '--r', linewidth=3)

            # plot sampled points (optional)
            ax.plot(x_front_bot, y_front_bot, 'b.', linewidth=1, label='sampled points')
            ax.plot(x_back_bot, y_back_bot, 'b.', linewidth=1)
            if not "sim" in fluid:
                ax.plot(x_front_top, y_front_top, 'b.', linewidth=1)
                ax.plot(x_back_top, y_back_top, 'b.', linewidth=1)
            
            # plot sampled outlier points (optional) 
            ax.plot(x_front_bot[~inlier_mask_front_bot], y_front_bot[~inlier_mask_front_bot], 'x', markeredgecolor='darkred', label='front outliers')
            ax.plot(x_back_bot[~inlier_mask_back_bot], y_back_bot[~inlier_mask_back_bot], '+', markeredgecolor='darkred', label='back outliers')
            if not "sim" in fluid:
                ax.plot(x_front_top[~inlier_mask_front_top], y_front_top[~inlier_mask_front_top], 'x', markeredgecolor='darkred')
                ax.plot(x_back_top[~inlier_mask_back_top], y_back_top[~inlier_mask_back_top], '+', markeredgecolor='darkred')

            # plot fits
            if fit_method == 'fit_x(y)':
                ax.plot(x_fit_front_bot[inlier_mask_front_bot], y_front_bot[inlier_mask_front_bot], 'r-', linewidth=3, label='RANSAC estimated curve')
                ax.plot(x_fit_back_bot[inlier_mask_back_bot], y_back_bot[inlier_mask_back_bot], 'r-', linewidth=3)
                if not "sim" in fluid:
                    ax.plot(x_fit_front_top[inlier_mask_front_top], y_front_top[inlier_mask_front_top], 'r-', linewidth=3)
                    ax.plot(x_fit_back_top[inlier_mask_back_top], y_back_top[inlier_mask_back_top], 'r-', linewidth=3)

            elif fit_method == 'fit_y(x)':
                ax.plot(x_front_bot[inlier_mask_front_bot], y_fit_front_bot[inlier_mask_front_bot],  'r-', linewidth=3, label='RANSAC estimated curve')
                ax.plot(x_back_bot[inlier_mask_back_bot], y_fit_back_bot[inlier_mask_back_bot],  'r-', linewidth=3)
                if not "sim" in fluid:
                    ax.plot(x_front_top[inlier_mask_front_top], y_fit_front_top[inlier_mask_front_top], 'r-', linewidth=3)
                    ax.plot(x_back_top[inlier_mask_back_top], y_fit_back_top[inlier_mask_back_top],  'r-', linewidth=3)

            # cosmetics
            ax.legend(fancybox=False, framealpha=1.0, facecolor=[0.8,0.8,0.8], edgecolor='k', loc='center right', 
                      title=f"case{case} image {img_name}", fontsize=10, bbox_to_anchor=(0.99,0.45)) 
           
            #ax.legend(fancybox=False, framealpha=1.0, facecolor=[0.8,0.8,0.8], edgecolor='k', loc='center right', title=f"{case} {img_name}", fontsize=10, bbox_to_anchor=(0.99,0.45)) # 
      
            # save image
            #fig.savefig(os.path.join(path_data_experiments, f"05_plots/{fluid}/contactAngle_{case}_{img_name[-8:-4]}_.png"), bbox_inches='tight', dpi=600)


# --------------- append data to lists for storing

    # interface bottom
    list_ca_ransac_front_bot.append(theta_ransac_front_bot_case)
    list_ca_ransac_back_bot.append(theta_ransac_back_bot_case)
    list_rms_dist_to_fit_front_bot_ransac.append(rms_dist_to_fit_ransac_front_bot_case)   
    list_rms_dist_to_fit_back_bot_ransac.append(rms_dist_to_fit_ransac_back_bot_case)
    
    # interface top (only for experimental images, becasue simulations are done only on bottom half of geometry)
    if not "sim" in fluid:
        list_ca_ransac_front_top.append(theta_ransac_front_top_case)    
        list_ca_ransac_back_top.append(theta_ransac_back_top_case)    
        list_rms_dist_to_fit_front_top_ransac.append(rms_dist_to_fit_ransac_front_top_case)   
        list_rms_dist_to_fit_back_top_ransac.append(rms_dist_to_fit_ransac_back_top_case)


**Store data**

In [ ]:
# create dataframes
if "sim" in fluid:
    df_measured_contactangles_front = pd.DataFrame(list(zip(list_ca_ransac_front_bot, list_rms_dist_to_fit_front_bot_ransac)), 
                                             index=cases, columns =['list_ca_ransac_bot', 'list_rms_dist_to_fit_front_bot_ransac'])
    df_measured_contactangles_back = pd.DataFrame(list(zip(list_ca_ransac_back_bot, list_rms_dist_to_fit_back_bot_ransac)), 
                                             index=cases, columns =['list_ca_ransac_bot','list_rms_dist_to_fit_back_bot_ransac'])

else:
    df_measured_contactangles_front = pd.DataFrame(list(zip(list_ca_ransac_front_top, list_ca_ransac_front_bot, list_rms_dist_to_fit_front_top_ransac,list_rms_dist_to_fit_front_bot_ransac)), 
                                             index=cases, columns =['list_ca_ransac_top', 'list_ca_ransac_bot','list_rms_dist_to_fit_front_top_ransac', 'list_rms_dist_to_fit_front_bot_ransac'])

    df_measured_contactangles_back = pd.DataFrame(list(zip(list_ca_ransac_back_top, list_ca_ransac_back_bot, list_rms_dist_to_fit_back_top_ransac, list_rms_dist_to_fit_back_bot_ransac)), 
                                             index=cases, columns =['list_ca_ransac_top', 'list_ca_ransac_bot','list_rms_dist_to_fit_back_top_ransac', 'list_rms_dist_to_fit_back_bot_ransac'])

# display dataframes
display(df_measured_contactangles_front.head())
display(df_measured_contactangles_back.head())

# save dataframes
df_measured_contactangles_front.to_csv(join(path_save, f"df_measured_contactangles_poly{fit_degree}_{fluid}_front.csv"), index_label='case')
df_measured_contactangles_back.to_csv(join(path_save, f"df_measured_contactangles_poly{fit_degree}_{fluid}_back.csv"), index_label='case')

## (iv) Interface tracking

**User input: selection of cases and general parameters**

In [ ]:
# choose fluid
fluid =  "03_novec"

# choose cases that should be considered
# if empty --> all cases are used. insert 1,2,3 to choose case 1,2,3
case_selection = []

# choose the step of processed images (1 --> every image is used)
step_images = 1

# choose which result to plot ("all", "last", "none") --> "all" leads to longer runime.
plot_pictures = "none"

# choose at which point to end the analysis.
# use "when_cavities_are_reached" for contact angle measurements, "at_final_image" for line tracking.
end_analysis = "at_final_image"

# choose paths
path_data_experiments = "../data_experiments"
path_df_postproc = join(path_data_experiments, "case_parameters.xlsx")
path_df_channel_edges = join(path_data_experiments, fluid, "02_channel_edges", "df_channel_edges.csv")
path_df_kmeans = join(path_data_experiments, fluid,f"df_kmeans_{fluid}.pickle")
path_images = join(path_data_experiments, fluid, "01_images_preprocessed")
path_save = join(path_data_experiments, fluid, "04_interface_tracking")

# load kmeans dataframe (load it only once since it can take some time)
df_kmeans = pd.read_pickle(path_df_kmeans)

# create directory for saving results and plots
os.makedirs(path_save, exist_ok=True)

**User input: select position of sample lines** \
For Novec, two positions are chosen

In [ ]:
pos_y1_mm = 0.1
pos_y2_mm = 0.7

**Run analysis**

In [ ]:
# set case selection based on user imput
if case_selection == []:
    cases = [int(file) for file in os.listdir(path_images) if len(file)<3]    
else: 
    cases = case_selection
print(f"selected cases: {cases}")
    
# initialize lists for results
list_ca_ransac_front_top, list_ca_ransac_front_bot, list_rms_dist_to_fit_front_top_ransac, list_rms_dist_to_fit_front_bot_ransac = ([] for i in range(4))
list_ca_ransac_back_top, list_ca_ransac_back_bot, list_rms_dist_to_fit_back_top_ransac,list_rms_dist_to_fit_back_bot_ransac = ([] for i in range(4))
x_front_top = []
x_front_bot = []
x_back_top = []
x_back_bot = []
time_range = []
list_u = []
list_pos_y_top = []
list_pos_y_bot = []

# loop over cases
for case in cases:
    print(f"Case {case}")
    
    # get case data
    x_start, x_cavities, y_channelEdge_bottom, y_channelEdge_top, framerate, selected_images = get_case_parameters(fluid, case, path_df_channel_edges, path_df_postproc, path_images, step_images, end_analysis)    
    print(f'\t number of selected images: {len(selected_images)}')

    # get velocity for storing in dataframe
    df_postproc =  pd.read_excel(path_df_postproc, sheet_name=fluid, index_col="case", skiprows=1) 
    u = df_postproc['u'][case]

    
    # scale sampling position from mm to px
    width_channel_mm = 2
    pos_y_top = int(y_channelEdge_bottom - pos_y1_mm / width_channel_mm * abs(y_channelEdge_top - y_channelEdge_bottom))
    pos_y_bot = int(y_channelEdge_bottom - pos_y2_mm / width_channel_mm * abs(y_channelEdge_top - y_channelEdge_bottom)) 
          
    # get inteerface mask for this case    
    mask_interface_case = df_kmeans['mask_interface'][case][::step_images]
    
    # initialize lists for storing
    list_idx_front_top = []
    list_idx_front_bot = []
    list_idx_back_top = []
    list_idx_back_bot = []
  
    # loop over images
    for i in range(len(selected_images)):
        #print(f'i: {i}')

        # ----- Get Image and interface mask

        # Load the image and convert it to grayscale
        img_name = selected_images[i]
        image = cv2.imread(os.path.join(path_images, str(case), img_name))
        image_gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
        
        # get interface mask
        mask_interface = mask_interface_case[i]
            
        # --------------- sample interface front and back at sample positions

        (idx_front_top, idx_back_top) = find_front_and_back_of_row_in_mask(mask_interface[pos_y_top-y_channelEdge_top])
        (idx_front_bot, idx_back_bot) = find_front_and_back_of_row_in_mask(mask_interface[pos_y_bot-y_channelEdge_top])

        list_idx_front_top.append(idx_front_top)
        list_idx_front_bot.append(idx_front_bot)
        list_idx_back_top.append(idx_back_top)
        list_idx_back_bot.append(idx_back_bot)

        # ---------------- plot

        plot_now = check_if_plot_now(plot_pictures, selected_images, i)

        if plot_now == True:

            # plot the picture
            fig, ax = plt.subplots(figsize=(10,10))
            plt.imshow(image_gray, cmap='gray', vmin=0, vmax=255)

            # plot the clusters
            X = np.argwhere(mask_interface)
            y = X[:, 0]#.reshape(-1, 1)
            x = X[:, 1]
            ax.plot(x+x_start, y+y_channelEdge_top, '.', label='interface cluster')

            # plot sample lines
            ax.plot([x_start,len(image_gray[0])-x_start] , [pos_y_top, pos_y_top], '--k', linewidth=3, label = 'sample lines')
            ax.plot([x_start,len(image_gray[0])-x_start] , [pos_y_bot, pos_y_bot], '--k', linewidth=3)
            
            # plot tracked pixels
            ax.plot(list_idx_front_top[i]+x_start, pos_y_top, '+r', ms=30, mew=4, label = 'front pixel')
            ax.plot(list_idx_front_bot[i]+x_start, pos_y_bot, '+r', ms=30, mew=4)
            ax.plot(list_idx_back_top[i]+x_start, pos_y_top, '+', color='orange', ms=30, mew=4, label = 'back pixel')
            ax.plot(list_idx_back_bot[i]+x_start, pos_y_bot, '+', color='orange', ms=30, mew=4)
            ax.legend(fancybox=False, framealpha=1.0, facecolor=[0.8,0.8,0.8], edgecolor='k', loc='center right', 
                      fontsize=10, bbox_to_anchor=(0.99,0.45), title=f"case {case}, img {img_name[-8:-4]}")
            
            # save figure
            #fig.savefig(join(path_save, f"interfaceTracking_{fluid}_{case}_{img_name[-8:-4]}.png"), bbox_inches='tight', dpi=300)            

    # normalize values
    x_front_top_case = np.array(list_idx_front_top) * 2e-3 / abs(y_channelEdge_top - y_channelEdge_bottom)
    x_front_bot_case = np.array(list_idx_front_bot) * 2e-3 / abs(y_channelEdge_top - y_channelEdge_bottom)
    x_back_top_case = np.array(list_idx_back_top) * 2e-3 / abs(y_channelEdge_top - y_channelEdge_bottom)
    x_back_bot_case = np.array(list_idx_back_bot) * 2e-3 / abs(y_channelEdge_top - y_channelEdge_bottom)
    time_range_case = range(len(x_front_top_case))/framerate

    # collect data for saving
    x_front_top.append(x_front_top_case)
    x_front_bot.append(x_front_bot_case)
    x_back_top.append(x_back_top_case)
    x_back_bot.append(x_back_bot_case)
    time_range.append(time_range_case)
    list_pos_y_top.append(pos_y_top)
    list_pos_y_bot.append(pos_y_bot)
    list_u.append(df_postproc['u'][case])
    
    # plot interface tracking result
    fig2, ax2 = plt.subplots(figsize=(5,5))
    ax2.plot(time_range_case, x_front_top_case,'+', label= 'front top')
    ax2.plot(time_range_case, x_front_bot_case,'+', label = 'front bottom')
    ax2.plot(time_range_case, x_back_top_case,'x', label= 'back top')
    ax2.plot(time_range_case, x_back_bot_case,'x', label = 'back bottom')

    # cosmetics
    ax2.set_xlabel('t (s)')
    ax2.set_ylabel('x (m)')
    ax2.legend()
    ax2.set_title(f"interface tracking result, case {case}")
    
    #save figure
    fig2.savefig(join(path_save, f"interfaceTrackingResult_{fluid}_{case}.png"), bbox_inches='tight', dpi=300)
    

**store data**

In [ ]:
# create dataframe
df_interface_tracking= pd.DataFrame(zip(list_u,list_pos_y_top, list_pos_y_bot,  time_range, x_front_top, x_front_bot, x_back_top, x_back_bot), 
                                    index=cases, columns =['u', 'y_top', 'y_bot','time_range', 'x_mm_front_top', 'x_mm_front_bot', 'x_mm_back_top', 'x_mm_back_bot'])
# display dataframe
display(df_interface_tracking)

# save dataframe
df_interface_tracking.to_csv(join(path_save, f"df_interface_tracking_{fluid}.csv"), index_label='case')